## Have Customers Narrow Their Travel Searches Based on Temperature and Precipitation

In [13]:
import pandas as pd
import os
import re
from config import g_key
import requests
import gmaps

Import the WeatherPy_vacation.csv file from Part 1 as a new DataFrame

In [14]:
# Input file
infolder = "weather_data"
infile = "WeatherPy_challenge.csv"

weather_data_df = pd.read_csv(os.path.join(infolder,infile))
weather_data_df

,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description,Rain (inches),Snow (inches)
0,Assiut,EG,27.18,31.18,89.60,29,0,17.22,clear sky,0.0,0.0
1,Salalah,OM,17.02,54.09,84.20,83,75,5.82,broken clouds,0.0,0.0
2,Nome,US,64.50,-165.41,42.01,93,90,8.05,overcast clouds,0.0,0.0
3,Grindavik,IS,63.84,-22.43,44.60,93,90,28.86,light rain,0.0,0.0
4,Hithadhoo,MV,-0.60,73.08,83.89,78,14,10.89,light rain,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...
555,Cabo San Lucas,MX,22.89,-109.91,81.00,47,5,6.93,clear sky,0.0,0.0
556,Nhulunbuy,AU,-12.23,136.77,69.80,83,0,3.36,clear sky,0.0,0.0
557,Mumbwa,ZM,-14.98,27.06,66.63,33,9,2.64,clear sky,0.0,0.0
558,Hasaki,JP,35.73,140.83,64.40,88,75,6.93,broken clouds,0.0,0.0


Filter the DataFrame for minimum and maximum temperature preferences, and if the rain or snow accumulation is 0 inches or not using conditional statements.

In [15]:
# Constants
yesno_regex = "^(yes|no)$"

In [16]:
#Prompt the customer for the minimum temperature preference.

# Keep prompting the user for the minimum temperature until a valid temperature is entered
while True:
    try:
        
        # Prompt the user for the minimum temperature
        min_str = input("\nWhat is the minimum temperature you would like for your trip? ")
        
        # Convert the entered string value to float
        pref_temp_min = float(min_str)
        
        # If the previous step did not cause an exception, exit the while loop
        break
    except:
        
        # If an exception was thrown, print an error message and recycle through the while loop
        print(f"\n<{min_str}> is an invalid temperature value. Please try again.")
        pass   


What is the minimum temperature you would like for your trip? xy

<xy> is an invalid temperature value. Please try again.

What is the minimum temperature you would like for your trip? 75


In [17]:
#Prompt the customer for the maximum temperature preference.

# Keep prompting the user for the maximum temperature until a valid temperature is entered
while True:
    try:
        
        # Prompt the user for the minimum temperature
        max_str = input("\nWhat is the maximum temperature you would like for your trip? ")
        
        # Convert the entered string value to float
        pref_temp_max = float(max_str)
        
        # If the previous step did not cause an exception, exit the while loop
        break
    except:
        
        # If an exception was thrown, print an error message and recycle through the while loop
        print(f"\n<{max_str}> is an invalid temperature value. Please try again.")
        pass


What is the maximum temperature you would like for your trip? yz

<yz> is an invalid temperature value. Please try again.

What is the maximum temperature you would like for your trip? 85


In [18]:
# Prompt the customer to answer if he or she would like it to be raining or not

# Keep asking the user if he or she would like it to be raining or not until a valid answer (yes or no) is entered
while True: 
    
    # Ask the user if he or she wants it to be raining
    raining = input("Do you want it to be raining? (yes/no) ")
    
    # Convert the answer to lower case
    raining = raining.lower()
    
    # If the answer matches the yesno_regex regular expression
    if re.search(yesno_regex, raining):
        
        # Set raining to the truth of (raining equals "yes")
        raining = raining == "yes"
        
        # Exit the while loop
        break
    else:
        
        # If the answer does not match the yesno_regex regular expression, print an error message 
        # and recyle through the while loop
        print("Please answer yes or no.")
    

Do you want it to be raining? (yes/no) yeah
Please answer yes or no.
Do you want it to be raining? (yes/no) no


In [19]:
# Prompt the customer to answer if he or she would like it to be snowing or not. 
# NOTE: It can either be raining or snowing, but not both


# If the answer to raining is no
# Keep asking the user if he or she would like it to be snowing or not until a valid answer (yes or no) is entered

if not raining:
    while True: 
        
        # Ask the user if he or she wants it to be raining
        snowing = input("Do you want it to be snowing? (yes/no) ")
        
        # Convert the answer to lower case
        snowing = snowing.lower()
        
        # If the answer matches the yesno_regex regular expression
        if re.search(yesno_regex, snowing):
            
            # Set snowing to the truth of (snowing equals "yes")
            snowing = snowing == "yes"
            
            # Exit the while loop
            break
        else:
            
            # If the answer does not match the yesno_regex regular expression, print an error message 
            # and recyle through the while loop
            print("Please answer yes or no.")
else:
    
    # if the answer to raining is "yes," then set snowing to False without prompting the user
    snowing = False


Do you want it to be snowing? (yes/no) nah
Please answer yes or no.
Do you want it to be snowing? (yes/no) no


In [20]:
# Filter dataframe 

# city_data_df contains cities whose Max Temp value falls within the range of pref_temp_min and pref_temp_max inclusive
city_data_df = weather_data_df.loc[(weather_data_df["Max Temp"] <= pref_temp_max) &
                              (weather_data_df["Max Temp"] >= pref_temp_min) ].copy().reset_index(drop=True)

if raining and not snowing:
    # If the answer to raining is yes (=> snowing is no), filter the city_data_df to cities 
    # where the inches of rain is greater than 0 and the inches of snow is equal to 0
    city_data_df = city_data_df[city_data_df["Rain (inches)"] > 0]
    city_data_df = city_data_df[city_data_df["Snow (inches)"] == 0]
elif snowing and not raining:
    # If the answer to raining is no and the answer to snowing is yes, filter the city_data_df to cities 
    # where the inches of rain is equal to 0 and the inches of snow is greater than 0
    city_data_df = city_data_df[city_data_df["Snow (inches)"] > 0]
    city_data_df = city_data_df[city_data_df["Rain (inches)"] == 0]
else:
    # If the answers to raining and snowing are both no, filter the city_data_df to cities 
    # where the inches of rain and inches of snow are both equal to 0
    city_data_df = city_data_df[city_data_df["Rain (inches)"] == 0]
    city_data_df = city_data_df[city_data_df["Snow (inches)"] == 0]
    

Code is written to search
for a hotel using a Google
API and the JSON data is
retrieved.

In [21]:
# This function retrieves the list of hotels corresponding to the lists of latitude and longitude that are passed in as arguments
# Arguments
# lat - list of city latitudes
# lng - list of city longitudes
#
# Return
# list of hotel json objects

def get_hotel(lat,lng):
    
    # Use base URL to search for hotels
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
    # params ... missing location
    params_temp = {"radius": 5000,
    "types": "lodging",
    "key": g_key}
    
    # Initialize the hotels list, the list of hotels to be returned
    hotels = []
    
    # Iterate through the matching lists of lat and lng
    for index in range(len(lat)):
        
        # refresh the params variable for each new city
        params = params_temp
        
        # set the params location to the (lat,lng) values of the current city
        params["location"] = f"{lat[index]},{lng[index]}"
            
        # Make request and get the JSON data from the search.
        hotels.append(requests.get(base_url, params=params).json())
        
    return hotels

In [22]:
# Create a new DataFrame with the following columns: ● City ● Country ● Max Temp ● Current Description ● Lat ● Lng
vacation_df = city_data_df[["City","Country","Max Temp","Current Description","Lat","Lng"]].copy().reset_index(drop=True)

# Pass the lists to the get_hotel function. Get a list of json payloads of hotel names for each location
hotels = get_hotel(vacation_df["Lat"],vacation_df["Lng"])

# Loop through the hotels json list and retrieve the first hotel name for each location 
hotels = pd.Series(hotels)
hotel_names = []

# Iterate through the retrieves hotels json payload
for hotel in hotels:
    # A try/except block is used to add the hotel to the new DataFrame, and the IndexError is resolved.
    try:
        # If a hotel name is found for the current city
        hotel_name = hotel["results"][0]["name"]
        
        # Add the new name to the hotel_names list
        hotel_names.append(hotel_name)
        
    except (IndexError):
        
        # If no hotels are found, add the value "NA" to the hotel_names list
        hotel_names.append("NA")

# Create the Hotel Name column in the hotels_df dataframe
vacation_df["Hotel Name"] = hotel_names

vacation_df

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Salalah,OM,84.20,broken clouds,17.02,54.09,HAMDAN PLAZA HOTEL
1,Hithadhoo,MV,83.89,light rain,-0.60,73.08,Scoop Guest House
2,San Policarpo,PH,82.80,broken clouds,12.18,125.51,Zal-Ann Beach Resort
3,Pran Buri,TH,82.18,overcast clouds,12.38,99.91,Waterside Resort
4,Gorontalo,ID,75.87,scattered clouds,0.54,123.06,Maqna Hotel by Prasanthi
...,...,...,...,...,...,...,...
132,Zaragoza,ES,84.99,clear sky,41.66,-0.88,Hotel Goya
133,Tiarei,PF,75.20,broken clouds,-17.53,-149.33,Le Rocher de Tahiti
134,Sherman,US,78.01,clear sky,33.64,-96.61,La Quinta Inn & Suites by Wyndham Sherman
135,Lamar,US,77.00,clear sky,33.67,-95.58,Kings Inn Paris TX


In [23]:
# Save and upload the new DataFrame as WeatherPy_vacation.csv

outfolder = "weather_data"
outfile = "WeatherPy_vacation.csv"

vacation_df.to_csv(os.path.join(outfolder,outfile),index=False)


Create a marker layer
map with a pop-up marker
for each city that includes:
● Hotel name
● City
● Country
● Current weather
description with the
maximum
temperature

In [24]:
# Add marker layer containing Hotel Name, City, Country, Current Weather, and Max Temp for each city.

# Rename vacation_df columns (replace spaces with underscores) for use with itertuples()
vacation_df.columns = ["City","Country","Max_Temp","Current_Description","Lat","Lng","Hotel_Name"]

# Set the GPS locations of the markers
locations = vacation_df[["Lat", "Lng"]]

# instantiate a figure object
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)

# Create the info box template
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{}</dd>
<dt>City</dt><dd>{}</dd>
<dt>Country</dt><dd>{}</dd>
<dt>Current Weather</dt><dd>{} and {} °F</dd>
</dl>
"""

# Create the hotel_info list for the marker layer
hotel_info = [info_box_template.format(row.Hotel_Name,\
                                       row.City,\
                                       row.Country,\
                                       row.Current_Description,\
                                       row.Max_Temp) \
              for index, row in enumerate(vacation_df.itertuples())]

# Create then add the marker layer to the Google map
marker_layer = gmaps.marker_layer(locations,info_box_content=hotel_info)
fig.add_layer(marker_layer)

# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))